# Implementación de un ChatBot con memoria

En esta práctica se implementa un ChatBot con memoria, haciendo uso de Gradio para la visualización de una interfaz gráfica, y de la API de OpenAI para responder las diferentes preguntas o cuestiones del usuario. 

Para implementar la memoria, se concatenan en un vector las preguntas y respuestas del usuario y del ChatGPT, de tal forma que con cada nueva pregunta del usuario, se incorpora también toda la conversación.

# Instalación de librerías

In [ ]:
!pip install openai==0.28
!pip install python-dotenv
!pip install gradio

# Importación de librerías

In [9]:
import gradio as gr
import openai
from dotenv import load_dotenv
import os

load_dotenv()

# Obtener las variables de entorno con los endpoints y claves.
openai.api_type = os.getenv("AZURE_OPENAI_API_TYPE")
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")

# Creación de un chatbot con memoria

In [ ]:
# Función para manejar las solicitudes del chatbot con memoria
def chatbot_with_memory(message, chat_history):
    # Convertir la historia de chat en un formato adecuado para OpenAI
    messages = [{"role": "system", "content": "Eres un asistente útil y amigable."}]
    
    for user_msg, bot_msg in chat_history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})
    
    # Añadir el nuevo mensaje del usuario
    messages.append({"role": "user", "content": message})
    
    # Solicitud a OpenAI para generar la respuesta
    response = openai.ChatCompletion.create(
        engine="gpt-4-turbo",  # Nombre del deployment
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )

    # Obtener la respuesta del asistente
    bot_response = response['choices'][0]['message']['content']
    
    # Actualizar la historia del chat con el nuevo par usuario-bot
    chat_history.append((message, bot_response))  # Añadir nuevo mensaje al final del historial
    
    # Devolver el historial actualizado
    return chat_history

In [ ]:
# Definir la interfaz de Gradio
def create_chat_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## Chatbot con Memoria usando OpenAI")
        
        # Definir los componentes del chat
        chatbot = gr.Chatbot(label="Chat con OpenAI")
        msg = gr.Textbox(label="Tu mensaje")
        clear = gr.Button("Limpiar Chat")
        
        # Estado del historial de chat
        state = gr.State([])  # Estado para almacenar el historial del chat
        
        # Acción para enviar mensaje
        def send_message(user_message, chat_history):
            # Obtener la respuesta del chatbot con la memoria acumulada
            chat_history = chatbot_with_memory(user_message, chat_history)
            
            # Actualizar el chatbot con el historial completo de la conversación
            return "", chat_history, chat_history
        
        # Conectar los componentes
        msg.submit(send_message, [msg, state], [msg, state, chatbot])
        clear.click(lambda: None, None, chatbot, queue=False)  # Limpiar el chat
        
    return demo

# Ejecución del chatbot

In [ ]:
# Ejecutar la interfaz
if __name__ == "__main__":
    demo = create_chat_interface()
    demo.launch()